In [ ]:
import numpy as np
import pandas as pd
from scipy.linalg import null_space
import math
from tabulate import tabulate

Функции

In [ ]:
#базовый датчик

def LFSR(length, y):
  p = 32
  M = 2**(p-1)
  a = 843314861
  c = 453816693
  M = pow(2, p - 1)
  random_sample = []
  for i in range(length):
      y = (a * y + c) % M
      random_sample.append(y / M)
  return random_sample

In [ ]:
#проверка матрицы интенсивности на корректность

def Q_correct(Q):
  errors = []
  row_summs = np.sum(Q, axis=1)
  Q_diag = np.diag(Q)
  Q_not_diag = Q - np.diag(np.diag(Q))

  if not np.allclose(row_summs, 0, rtol = 1e-12):
    errors.append("Построчная сумма элементов матрицы интенсивности должна быть равна 0")

  if not np.all(Q_diag < 0):
    errors.append("Диагональные элементы матрицы интенсивности должны быть отрицательными")

  if not np.all(Q_not_diag >= 0):
    errors.append("Недиагональные элементы матрицы интенсивности должны быть >= 0")

  if errors:
    raise ValueError("\n".join(errors))
  return True

In [ ]:
#функция нахождения вектора финальных вероятностей

def Pi(Q):
  pi = null_space(Q.T).flatten()
  pi = pi / np.sum(pi)
  return pi

In [ ]:
#Функция генерирования Марковской цепи

def Markov_chain_gen(Q, N):

  I=[x for x in range(len(Q))]

  i = np.random.choice(I, p=Pi(Q))
  t = 0
  times = []
  states = []

  for x in range(N):
    times.append(t)
    a = LFSR(1, 12345454+x)
    tao = np.log(a[0])/Q[i][i]
    t = t + tao

    p = []
    next = []

    for j in range(len(Q[i])):
      if i == j:
        continue
      else:
        p.append(-Q[i][j]/Q[i][i])
        next.append(j)

    states.append(i)
    j = np.random.choice(next, p = p)
    i = j

  return states, times

In [ ]:
#Функция вычисления суммарного времени проведенного в каждом из состояний
def Sum_time(states, times):

  states_dictionary = dict.fromkeys(states, 0)

  for i in range(N-1):
    states_dictionary[states[i]] += times[i+1]-times[i]

  return states_dictionary

In [ ]:
#Функция нормирования суммарного времени

def Norm(states_dictionary):
  norm_states_dictionary = states_dictionary
  sum_d = sum(norm_states_dictionary.values())

  for i in states_dictionary.keys():
    norm_states_dictionary[i] = norm_states_dictionary[i]/sum_d

  return norm_states_dictionary

In [ ]:
#Функция сравнения финальных вероятностей(pi) с относительным временем

def Pi_and_norm_comparison(Q, norm_states_dictionary):
  print()
  arr = Pi(Q)

  for i in range(len(Pi(Q))):
    print(arr[i],norm_states_dictionary[i])

In [ ]:
#Финальная функция

def f(Q,N):

  Q_correct(Q)

  states, times = Markov_chain_gen(Q, N)
  print(f"\n----------------------------------------------\n{N} переходов цепи из состояния в состояние:\n")
    # Выводим таблицу сравнений
  table = [["Строка", "Позиция", "Время"]]

  for i, (state, time) in enumerate(zip(states[:5], times[:5]), 1):
      table.append([i, state, time])

  if len(states) > 10:
      table.append(["...", "...", "..."])

  for i, (state, time) in enumerate(zip(states[-5:], times[-5:]), len(states) - 4):
      table.append([i, state, time])

  print(tabulate(table, headers="firstrow"))

  states_dictionary = Sum_time(states, times)


  print("\n----------------------------------------------\nСуммарное время пребывания цепи в каждом состоянии:")

  print("\n", {int(k): float(v) for k, v in states_dictionary.items()})

  print("\nСумма:\n", sum(states_dictionary.values()))

  norm_states_dictionary = Norm(states_dictionary)
  print("\n----------------------------------------------\nОтносительное время пребывания в каждом состоянии:\n\n", {int(k): float(v) for k, v in norm_states_dictionary.items()})

  print("\n----------------------------------------------\nСравнение относительного времени с финальными вероятностями:")
  Pi_and_norm_comparison(Q, norm_states_dictionary)

Реализация

In [ ]:
N = 10000

In [ ]:
# #задаем матрицу интенсивности

# Q = np.array([
# [-3, 2, 1],
# [0.5, -2, 1.5],
# [3, 0, -3]])

In [ ]:
Q = np.array([[-4, 1, 2, 1],
              [8, -24, 8, 8],
              [7, 8, -23, 8],
              [1, 1, 1, -3]]) # Матрица инфинитезимальных коэффициентов

In [ ]:
f(Q,N)


----------------------------------------------
10000 переходов цепи из состояния в состояние:

Строка    Позиция    Время
--------  ---------  -------------------
1         0          0
2         3          0.09423371899114655
3         2          0.9417746153388618
4         3          0.9744761567648585
5         2          1.0231812857887757
...       ...        ...
9996      3          1618.6144895286404
9997      1          1618.72710580233
9998      0          1618.8206181119565
9999      2          1618.9945551100375
10000     0          1618.9995534407424

----------------------------------------------
Суммарное время пребывания цепи в каждом состоянии:

 {0: 606.5069957685307, 3: 790.7477053668847, 2: 119.83251427165632, 1: 101.91233803367064}

Сумма:
 1618.9995534407424

----------------------------------------------
Относительное время пребывания в каждом состоянии:

 {0: 0.3746183836058295, 3: 0.48841749442510096, 2: 0.07401639735908819, 1: 0.06294772460998135}

----------